In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch


In [112]:
l = [3,4,5]
t = [4,8,12]
for i in l:
    for j in t:
        fpath = f'explain_res/ogbg_molhiv/GIN_layer_{i}/res_dict_top{j}.pt'
        res = torch.load(fpath,map_location=torch.device('cpu'))
        print (f'layer {i} top {j}')
        for k in res:
            print (res[k][1])

layer 3 top 4
auc1:0.8490844964981079,auc2:0.7870215177536011,auc-gap:0.062062978744506836,fidelity:0.013439144007861614
layer 3 top 8
auc1:0.8492156267166138,auc2:0.7340103387832642,auc-gap:0.11520528793334961,fidelity:0.016068918630480766
layer 3 top 12
auc1:0.8518242835998535,auc2:0.7112181186676025,auc-gap:0.14060616493225098,fidelity:0.017382413148880005
layer 4 top 4
auc1:0.8360274434089661,auc2:0.7975471019744873,auc-gap:0.03848034143447876,fidelity:0.012618200853466988
layer 4 top 8
auc1:0.8363205194473267,auc2:0.774978756904602,auc-gap:0.06134176254272461,fidelity:0.014235962182283401
layer 4 top 12
auc1:0.838350772857666,auc2:0.7550452947616577,auc-gap:0.0833054780960083,fidelity:0.01645577698945999
layer 5 top 4
auc1:0.8388746380805969,auc2:0.7851077914237976,auc-gap:0.053766846656799316,fidelity:0.017300616949796677
layer 5 top 8
auc1:0.8391385078430176,auc2:0.7420946955680847,auc-gap:0.09704381227493286,fidelity:0.01970428228378296
layer 5 top 12
auc1:0.842008113861084,auc

In [11]:
key = 8
fpath = f'explain_res/ogbg_molbbbp/GIN_layer_4/res_dict_top{key}.pt'
res = torch.load(fpath,map_location=torch.device('cpu'))

N = len(res[key][0])
print (res[key][1])
val1=0
val2 = 0
val3 = 0
for i in range(N):
    v1 = torch.sum(res[key][0][i][1])
    v2 = torch.sum(res[key][0][i][2])
    v3 = torch.sum(res[key][0][i][3])
    if v1>0:
        val1+=1
    if v2>0:
        val2+=1
    if v3>0:
        val3+=1

print (val1,val2,val3,N)

auc1:0.9414114952087402,auc2:0.8763255476951599,auc-gap:0.06508594751358032,fidelity:0.1892925500869751
589 347 202 1631


In [12]:
for k in res:
    print (k)

8


In [14]:
res[8][0]

[[tensor([ 1,  2,  3,  4, 14, 19]),
  tensor([1., 1., 1., 1., 1., 1.]),
  tensor([0., 0., 0., 0., 0., 0.]),
  tensor([0., 0., 0., 0., 0., 0.])],
 [tensor([0, 2, 3, 4, 5, 6]),
  tensor([0., 0., 0., 0., 0., 0.]),
  tensor([0., 0., 0., 0., 0., 0.]),
  tensor([0., 0., 0., 0., 0., 0.])],
 [tensor([14, 15, 16, 17, 18]),
  tensor([0., 0., 0., 0., 0.]),
  tensor([0., 0., 0., 0., 0.]),
  tensor([0., 0., 0., 0., 0.])],
 [tensor([ 6,  7, 17, 18, 19, 20]),
  tensor([0., 0., 0., 0., 0., 0.]),
  tensor([0., 0., 0., 0., 0., 0.]),
  tensor([0., 0., 0., 0., 0., 0.])],
 [tensor([19, 22, 23, 26, 27]),
  tensor([0., 0., 0., 0., 0.]),
  tensor([0., 0., 0., 0., 0.]),
  tensor([0., 0., 0., 0., 0.])],
 [tensor([0, 1, 2, 3, 4]),
  tensor([0., 0., 0., 0., 0.]),
  tensor([0., 0., 0., 0., 0.]),
  tensor([0., 0., 0., 0., 0.])],
 [tensor([1, 2, 3, 4, 5]),
  tensor([1., 1., 1., 1., 1.]),
  tensor([1., 1., 1., 1., 1.]),
  tensor([0., 0., 0., 0., 0.])],
 [tensor([ 1,  3, 12, 13, 15, 16]),
  tensor([0., 0., 0., 0., 0.,

In [3]:
import torch

# ---------------------------------------------------------------------
# 1) Define a mock PyG Batch-like object:
#    We'll create a simple class that holds 'edge_index' and 'edge_gt'.
# ---------------------------------------------------------------------
class MockBatch:
    def __init__(self, edge_index, edge_gt):
        """
        Simulates a minimal batch object from PyG with:
          - edge_index: Tensor(2, E)
          - edge_gt: Boolean Tensor(E,)
        """
        self.edge_index = edge_index
        self.edge_gt = edge_gt

# ---------------------------------------------------------------------
# 2) Create some sample data (edges + ground-truth flags).
#    Some edges are reversed duplicates to test 'undirected' logic.
# ---------------------------------------------------------------------
edge_index = torch.tensor([
    [0, 1, 1, 2, 3, 2],  # row for 'source' node
    [1, 0, 2, 1, 2, 3]   # row for 'target' node
], dtype=torch.long)

# Suppose edges 0->1 and 1->0 are both marked as True, similarly for 1->2 and 2->1
# We'll mark them to see how unique counting works.
edge_gt = torch.tensor([True, True, True, True, True, False], dtype=torch.bool)

# Create the mock batch
batch = MockBatch(edge_index, edge_gt)

# ---------------------------------------------------------------------
# 3) Use the get_gt_edges_undirected function.
#    (Assume it's already defined or imported.)
# ---------------------------------------------------------------------
def get_gt_edges_undirected(batch):
    """
    Processes a batch from a PyG dataloader to:
      1) Extract ground-truth edges (with duplication).
      2) Compute the number of unique ground-truth edges assuming undirected edges.

    Parameters:
        batch: A PyG Batch object with the following attributes:
            - batch.edge_index: Tensor of shape (2, E), representing edge indices.
            - batch.edge_gt: Tensor of shape (E,), containing boolean values indicating ground truth edges.

    Returns:
        tuple:
            - num_unique (int): The number of unique ground-truth edges (undirected).
            - gt_edge_index (Tensor): All ground-truth edge indices (2, E_gt), possibly with duplicates.
    """

    # 1) Get the indices where batch.edge_gt == True
    true_indices = torch.nonzero(batch.edge_gt, as_tuple=False).squeeze()

    # 2) Collect the ground-truth edges with duplication
    gt_edge_index = batch.edge_index[:, true_indices]
    print (gt_edge_index)
    # 3) Sort each edge so that (u, v) has u < v if it is undirected
    sorted_gt_edge_index = torch.sort(gt_edge_index, dim=0).values
    print (sorted_gt_edge_index)
    # 4) Extract unique edges (column-wise uniqueness) for undirected edges
    unique_sorted_edges = torch.unique(sorted_gt_edge_index, dim=1)
    print (unique_sorted_edges)
    # 5) Compute the number of unique ground-truth edges
    num_unique = unique_sorted_edges.size(1)

    # Return the number of unique edges and the full set of edges (with duplicates)
    return num_unique, gt_edge_index

# ---------------------------------------------------------------------
# 4) Run the function on our test batch
# ---------------------------------------------------------------------
num_unique, gt_edges = get_gt_edges_undirected(batch)

print("All GT edges (with duplicates):")
print(gt_edges)
print("Number of unique GT edges (undirected):", num_unique)

# ---------------------------------------------------------------------
# Expected behavior:
#   - The function should return:
#       a) The number of unique undirected edges
#       b) The entire list of GT edges (with duplicates)
#
#   - With the sample data, edges are:
#       (0,1), (1,0), (1,2), (2,1), (3,2) as GT (the last is not, edge_gt = False)
#     After sorting each column:
#       (0,1)  (0,1)  (1,2)  (1,2)  (2,3)
#     Unique edges among them = (0,1), (1,2), (2,3).
#     So num_unique = 3.
# ---------------------------------------------------------------------


tensor([[0, 1, 1, 2, 3],
        [1, 0, 2, 1, 2]])
tensor([[0, 0, 1, 1, 2],
        [1, 1, 2, 2, 3]])
tensor([[0, 1, 2],
        [1, 2, 3]])
All GT edges (with duplicates):
tensor([[0, 1, 1, 2, 3],
        [1, 0, 2, 1, 2]])
Number of unique GT edges (undirected): 3
